In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
#show plots inline
%matplotlib inline
import statsmodels.api as sm
from scipy import stats

C:\Users\mrb14\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
students = pd.read_csv("MultiVariateCumGPA.csv",header=0,sep=',')

In [127]:

from geopy.geocoders import Nominatim
from geopy.distance import vincenty

cnt = 0
NewYork = (40.7306458, -73.9866136)

for index,row in students.iterrows():
    if (np.isnan(students.iloc[index]['distance']) ) and cnt < 1000:
        
        try:
            geolocator = Nominatim()
            location = geolocator.geocode(row['City'] + ',' + row['StateProvince'] + ',' + row['country'] )
            geolocation = (location.latitude , location.longitude)
            students.ix[index,'distance'] = vincenty(geolocation,NewYork).miles
        except:
            students.ix[index,'distance'] = 0
        cnt +=1
students.to_csv("MultiVariateCumGPA.csv")
    

In [8]:
#sns.pairplot(students, size=4)
#plt.show()

In [3]:
list(students.columns.values)

['cnt',
 'entryId',
 'CUM_GPA',
 '(No column name)',
 'AvgCredits',
 'gender',
 'ProfileWeightEnum',
 'City',
 'StateProvince',
 'country',
 'distance']

In [124]:
from itertools import combinations
from decimal import *
from IPython.display import display, HTML

tmpdf = students.drop('CUM_GPA',1)
tmpdf = tmpdf.drop('City',1)
tmpdf = tmpdf.drop('StateProvince',1)
tmpdf = tmpdf.drop('country',1)
tmpdf = tmpdf.drop('distance',1)
tmpdf = tmpdf.drop('(No column name)',1)

combos = list(combinations(tmpdf.columns.values,2)) + list(combinations(tmpdf.columns.values,3)) + list(combinations(tmpdf.columns.values,4))
i = 0

print(list(combinations(tmpdf.columns.values,4)))
print('hello')
print(list(combinations(tmpdf.columns.values,3)))
summaryDF = pd.DataFrame(columns=['students','variables','rsquared','rsquared_adj','C-p','aic','aic_c','predictors'])

for combo in combos:
    X = students[list(combo)]
    X = sm.add_constant(X)
    model = sm.OLS(students['CUM_GPA'],X ).fit()
    print(model.std())
    ssr = round(Decimal(model.ssr),6)
    mse = round(Decimal(model.mse_resid),6)
    cp = round(ssr/mse,6) - len(students) + 2*len(list(combo)) + 2
    aic_c = model.aic + 2*(len(list(combo))+2)*(len(list(combo))+3)/(len(students)-len(list(combo))-3)
    summaryDF = pd.concat([summaryDF,pd.DataFrame({'students':len(students),'variables':len(list(combo)), 'rsquared':model.rsquared, 'rsquared_adj':model.rsquared_adj, 'C-p':cp,'aic':model.aic,'aic_c':aic_c,'predictors':str(list(combo))},index=[i])])   
    i = i+1
    
    
for combo in tmpdf.columns.values:
    X = students[combo]
    X = sm.add_constant(X)
    model = sm.OLS(students['CUM_GPA'],X ).fit()
    ssr = round(Decimal(model.ssr),6)
    mse = round(Decimal(model.mse_resid),6)
    cp = round(ssr/mse,6) - len(students) + 2*1 + 2
    aic_c = model.aic + 2*(1+2)*(1+3)/(len(students)-1-3)
    summaryDF = pd.concat([summaryDF,pd.DataFrame({'students':len(students),'variables':1, 'rsquared':model.rsquared, 'rsquared_adj':model.rsquared_adj, 'C-p':cp,'aic':model.aic,'aic_c':aic_c,'predictors':combo},index=[i])])   
    i = i+1
display(HTML(summaryDF[['students','variables','rsquared','rsquared_adj','C-p','aic','aic_c','predictors']].sort_values(by=['variables','rsquared']).to_html()))

    

[('cnt', 'entryId', 'AvgCredits', 'gender'), ('cnt', 'entryId', 'AvgCredits', 'ProfileWeightEnum'), ('cnt', 'entryId', 'gender', 'ProfileWeightEnum'), ('cnt', 'AvgCredits', 'gender', 'ProfileWeightEnum'), ('entryId', 'AvgCredits', 'gender', 'ProfileWeightEnum')]
hello
[('cnt', 'entryId', 'AvgCredits'), ('cnt', 'entryId', 'gender'), ('cnt', 'entryId', 'ProfileWeightEnum'), ('cnt', 'AvgCredits', 'gender'), ('cnt', 'AvgCredits', 'ProfileWeightEnum'), ('cnt', 'gender', 'ProfileWeightEnum'), ('entryId', 'AvgCredits', 'gender'), ('entryId', 'AvgCredits', 'ProfileWeightEnum'), ('entryId', 'gender', 'ProfileWeightEnum'), ('AvgCredits', 'gender', 'ProfileWeightEnum')]


AttributeError: 'OLSResults' object has no attribute 'std'

In [123]:
TrimmedDF = pd.DataFrame(columns=['students','variables','rsquared','rsquared_adj','C-p','aic','aic_c','predictors'])

for x in range(1, 5):
    tmp = summaryDF.loc[summaryDF['variables'] == x].sort_values(by=['variables','rsquared'],ascending=[True,False])
    for y in range (0,2):
        tmp2 = tmp.iloc[y].to_frame()
        
        TrimmedDF = pd.concat([TrimmedDF,pd.DataFrame({'students':tmp2.loc['students'].item(),'variables':tmp2.loc['variables'].item(), 'rsquared':tmp2.loc['rsquared'].item(), 'rsquared_adj':tmp2.loc['rsquared_adj'].item(), 'C-p':tmp2.loc['C-p'].item(),'aic':tmp2.loc['aic'].item(),'aic_c':tmp2.loc['aic_c'].item(),'predictors':tmp2.loc['predictors'].item()},index=[i])])   
        #display(HTML(tmp2[['students','variables','rsquared','rsquared_adj','C-p','aic','aic_c','predictors']].to_html()))

display(HTML(TrimmedDF[['students','variables','rsquared','rsquared_adj','C-p','aic','aic_c','predictors']].sort_values(by=['variables','rsquared']).to_html()))
       
del TrimmedDF

,students,variables,rsquared,rsquared_adj,C-p,aic,aic_c,predictors
30,3319,1,0.025182,0.024888,2.008164,3168.384895,3168.392135,gender
30,3319,1,0.031884,0.031592,2.006273,3145.486382,3145.493622,AvgCredits
30,3319,2,0.033782,0.033199,2.995793,3140.973989,3140.986059,"['cnt', 'AvgCredits']"
30,3319,2,0.056077,0.055508,3.007166,3063.492179,3063.504249,"['AvgCredits', 'gender']"
30,3319,3,0.057382,0.056529,4.002053,3060.899713,3060.917824,"['cnt', 'AvgCredits', 'gender']"
30,3319,3,0.058045,0.057192,3.992849,3058.565233,3058.583344,"['AvgCredits', 'gender', 'ProfileWeightEnum']"
30,3319,4,0.058056,0.056919,5.006727,3060.526133,3060.551495,"['entryId', 'AvgCredits', 'gender', 'ProfileWe..."
30,3319,4,0.059297,0.058161,4.992521,3056.151714,3056.177077,"['cnt', 'AvgCredits', 'gender', 'ProfileWeight..."
